# preferably run this script on the grid

Based on Anne Urai's script fitHDDM.py


Changes:
1. Parameter estimations in dyadic case
2. Flexible handling of no. of lagged/previous trials tested for history effect
3. Encoding specific to our experiment
4. Simplified dataset handling

In [133]:
import glob
import hddm
import kabuki
import numpy as np
import os
import pandas as pd
import time

In [134]:
def recode_4stimcoding(mydata):
    #code stimulus and response direction left as 0, leave direction right as 1.
    
    mydata.loc[mydata['direction']==-1,'direction'] = 0
    mydata.loc[mydata['response']==-1,'response'] = 0
    for col in mydata.columns.tolist():
        if ('stim' in col) or ('resp' in col):
            mydata.loc[mydata[col]==-1,col] = 0
    
    return mydata

def z_link_func(x):
    return 1 / (1 + np.exp(-(x.values.ravel())))

def aic(self):
    k = len(self.get_stochastics())
    logp = sum([x.logp for x in self.get_observeds()['node']])
    return 2 * k - 2 * logp


def bic(self):
    k = len(self.get_stochastics())
    n = len(self.data)
    logp = sum([x.logp for x in self.get_observeds()['node']])
    return -2 * logp + k * np.log(n)

def concat_models(mypath, model_name, nchains=30):
    traces = range(1,nchains+1)

    # CHECK IF COMBINED MODEL EXISTS
    if os.path.isfile(os.path.join(mypath, model_name, 'modelfit-combined.model')):
        print("Combined Model exists: {}".format(os.path.join(mypath, model_name, 'modelfit-combined.model')))
    else:
        # ============================================ #
        # APPEND MODELS
        # ============================================ #
        allmodels = []
        print("Combining all traces for %s" % model_name)
        for trace_id in traces:  # how many chains were run?
            model_filename = os.path.join(mypath, model_name, 'modelfit-md%d.model' % trace_id)
            if os.path.isfile(model_filename) == True:  # if not, this model has to be rerun
                print(model_filename)
                thism = hddm.load(model_filename)
                allmodels.append(thism)  # now append into a list
            else:
                print("Not found: trace_id {:2d}".format(trace_id))
                
        if len(allmodels) != nchains:
            return None
        # ============================================ #
        # CHECK CONVERGENCE if all traces were found
        # ============================================ #    
        print("Performing gelman rubin convergence test\n")
        try:
            gr = hddm.analyze.gelman_rubin(allmodels)
            # save
            text_file = open(os.path.join(mypath, model_name, 'gelman_rubin.txt'), 'w')
            for p in gr.items():
                text_file.write("%s,%s\n" % p)
                # print a warning when non-convergence is detected
                # Values should be close to 1 and not larger than 1.02 which would indicate convergence problems.
                # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3731670/
                if abs(p[1] - 1) > 0.02:
                    print("non-convergence found, %s:%s" % p)
            text_file.close()
            print("written gelman rubin stats to file")
        except Exception as e:
            print("Error: {}".format(e))
        m = kabuki.utils.concat_models(allmodels) #creates one model from all chains

        # ============================================ #
        # SAVE THE FULL MODEL
        # ============================================ #

        m.save(os.path.join(mypath, model_name, 'modelfit-combined.model'))  # save combined modelto disk
        print("Concatenated model saved!")
        
        # ============================================ #
        # SAVE POINT ESTIMATES
        # ============================================ #

        print("saving stats...")
        results = m.gen_stats()  # point estimate for each parameter and subject
        results.to_csv(os.path.join(mypath, model_name, 'results-combined.csv'))

        # save the DIC for this model
        text_file = open(os.path.join(mypath, model_name, 'DIC-combined.txt'), 'w')
        text_file.write("Combined model: {}\n".format(m.dic))
        text_file.close()
        print('done')
        
        # ============================================ #
        # SAVE TRACES
        # ============================================ #

        print("saving traces...")
        # get the names for all nodes that are available here
        group_traces = m.get_group_traces()
        group_traces.to_csv(os.path.join(mypath, model_name, 'group_traces.csv'))

        all_traces = m.get_traces()
        all_traces.to_csv(os.path.join(mypath, model_name, 'all_traces.csv'))
        print('done')
        
        # ============================================ #
        # CONCATENATE MODEL COMPARISON
        # ============================================ #
        # average model comparison values across chains
        print('concatenating model comparison...')
        fls = glob.glob(os.path.join(mypath, model_name, 'model_comparison_md*.csv'))
        tmpdf = pd.concat([pd.read_csv(f) for f in fls ])
        # average over chains
        df2 = tmpdf.mean()
        df2 = tmpdf.describe().loc[['mean']]
        df2.to_csv(os.path.join(mypath, model_name, 'model_comparison.csv')) # save comparison to disk
        print('done')


        # DELETE FILES to save space
        print("Now deleting files for seperate chains...")
        for fl in glob.glob(os.path.join(mypath, model_name, 'modelfit-md*.model')):
            print(fl)
            os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'modelfit-md*.db')):
            if not '-md1.db' in fl: #needed here to load the pickled db in PPC
                print(fl)
                os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'model_comparison_md*.csv')):
            print(fl)
            os.remove(fl)
        for fl in glob.glob(os.path.join(mypath, model_name, 'DIC-md*.txt')):
            print(fl)
            os.remove(fl)
        print('DONE!!!')

In [135]:
def make_model(mydata, model_name, trace_id, nlag=0):
    
    #checks before model is created
    if "nohist" not in model_name and nlag == 0:
        print("For all models with history effects, 'nlag' must be non-zero")
        exit(0)
    elif "nohist" in model_name and nlag != 0:
        print("'nlag' specified but model is without history effect. 'nlag' value is ignored\n")
        nlag = 0
         
    if 'regress' in model_name:
        if nlag != 0:
            lags = range(1,nlag+1)
            resp_cols = ['l' + str(i) + '_resp' for i in lags]
            stim_cols = ['l' + str(i) + '_stim' for i in lags]
            resps = " + ".join(resp_cols)
            respstim = " + ".join(resp_cols + stim_cols)
            for col in stim_cols:
                mydata = mydata[mydata[col].notna()]
    else:
        mydata = recode_4stimcoding(mydata)
        if nlag != 0:
            col_resp = 'l' + str(nlag) + '_resp'
            col_stim = 'l' + str(nlag) + '_stim'
            col_stim = 'l' + str(nlag) + '_subject'
            col_repeat = 'l' + str(nlag) + '_repeat'
             
    if model_name == 'stimcoding_nohist': # NO HISTORY FOR MODEL COMPARISON
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'])
    elif model_name == 'stimcoding_nohist_svgroup': # DOES DRIFT RATE VARIABILITY REDUCE? GROUP
        # add a group to indicate bias magnitude
        sjrepetition = mydata.groupby(['subj_idx'])['l1_repeat'].mean().reset_index()
        sjrepetition['biasgroup'] = pd.qcut(np.abs(sjrepetition['l1_repeat'] - 0.5), 3, labels=False)
        mydata2 = pd.merge(mydata, sjrepetition, on='subj_idx', how='inner')
        m = hddm.HDDMStimCoding(mydata2, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sz'],
                depends_on={'sv': ['biasgroup']})
    elif model_name == 'stimcoding_dc_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp]})
    elif model_name == 'stimcoding_z_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'z':[col_resp]})
    elif model_name == 'stimcoding_dc_z_resp':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp], 'z':[col_resp]})
    elif model_name == 'stimcoding_dc_stim': # STIMCODING PREVSTIM
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_stim]})
    elif model_name == 'stimcoding_z_stim':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'z':[col_stim]})
    elif model_name == 'stimcoding_dc_z_stim':
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_stim], 'z':[col_stim]})
    elif model_name == 'stimcoding_dc_z_st_resp': # also estimate across-trial variability in nondecision time
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz', 'st'), group_only_nodes=['sv', 'sz', 'st'],
                depends_on={'dc':[col_resp], 'z':[col_resp]})
    elif model_name == 'stimcoding_dc_z_resp_svgroup':
        # add a group to indicate bias magnitude
        sjrepetition = mydata.groupby(['subj_idx'])[col_repeat].mean().reset_index()
        sjrepetition['biasgroup'] = pd.qcut(np.abs(sjrepetition[col_repeat] - 0.5), 3, labels=False)
        mydata2 = pd.merge(mydata, sjrepetition, on='subj_idx', how='inner')
        m = hddm.HDDMStimCoding(mydata2, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sz'],
                depends_on={'dc':[col_resp], 'z':[col_resp], 'sv': ['biasgroup']})
    elif model_name == 'stimcoding_dc_z_resp_groupsplit':# SEPARATE FIT FOR REPEATERS AND ALTERNATORS
        # add coding for repeaters and alternators
        sjrepetition = mydata.groupby(['subj_idx'])[col_repeat].mean().reset_index()
        sjrepetition['group'] = np.sign(sjrepetition[col_repeat] - 0.5)
        mydata2 = pd.merge(mydata, sjrepetition, on='subj_idx', how='inner')
        m = hddm.HDDMStimCoding(mydata2, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp, 'group'], 'z':[col_resp, 'group']})
    elif model_name == 'stimcoding_dc_z_resp_congruency': #SPLIT BY CONGRUENCE BETWEEN PREVIOUS CHOICE AND CURRENT STIMULUS
        # compute a double (not boolean) to indicate congruence
        mydata['congruent'] = (mydata[col_resp] == mydata['direction']) * 1
        m = hddm.HDDMStimCoding(mydata, stim_col='direction', split_param='v',
                drift_criterion=True, bias=True, p_outlier=0.05,
                include=('sv', 'sz'), group_only_nodes=['sv', 'sz'],
                depends_on={'dc':[col_resp, 'congruent'], 'z':[col_resp, 'congruent']})
    # ============================================ #
    # REGRESSION MODELS WITH MULTIPLE LAGS
    # ============================================ #
    elif model_name == 'regress_nohist':
        # only stimulus dependence
        v_reg = {'model': 'v ~ 1 + direction', 'link_func': lambda x:x}
        # specify that we want individual parameters for all regressors, see email Gilles 22.02.2017
        m = hddm.HDDMRegressor(mydata, v_reg,
            include=['z', 'sv'], group_only_nodes=['sv'],
            group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_dc_resp': #only previous response
        v_reg = {'model': 'v ~ 1 + direction + ' + resps, 'link_func': lambda x:x} 
        m = hddm.HDDMRegressor(mydata, v_reg,
            include=['z', 'sv'], group_only_nodes=['sv'],
            group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_z_resp': #only prevresp
        z_reg = {'model': 'z ~ 1  + ' + resps, 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + direction', 'link_func': lambda x:x}
        m = hddm.HDDMRegressor(mydata, [z_reg, v_reg],
            include=['z', 'sv'], group_only_nodes=['sv'],
            group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_dcz_resp': #only prevresp
        v_reg = {'model': 'v ~ 1 + direction + ' + resps, 'link_func': lambda x:x}
        z_reg = {'model': 'z ~ 1  + ' + resps, 'link_func': z_link_func}
        m = hddm.HDDMRegressor(mydata, [v_reg, z_reg],
                               include=['z', 'sv'], group_only_nodes=['sv'],
                               group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_dc_resp_stim': #both prevresp and prevstim
        v_reg = {'model': 'v ~ 1 + direction + ' + respstim, 'link_func': lambda x:x}
        m = hddm.HDDMRegressor(mydata, v_reg,
            include=['z', 'sv'], group_only_nodes=['sv'],
            group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_z_resp_stim':
        z_reg = {'model': 'z ~ 1  + ' + respstim, 'link_func': z_link_func}
        v_reg = {'model': 'v ~ 1 + direction', 'link_func': lambda x:x}
        m = hddm.HDDMRegressor(mydata, [z_reg, v_reg],
            include=['z', 'sv'], group_only_nodes=['sv'],
            group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    elif model_name == 'regress_dcz_resp_stim':
        v_reg = {'model': 'v ~ 1 + direction + ' + respstim, 'link_func': lambda x:x}
        z_reg = {'model': 'z ~ 1  + '+ respstim, 'link_func': z_link_func}
        m = hddm.HDDMRegressor(mydata, [v_reg, z_reg],
                               include=['z', 'sv'], group_only_nodes=['sv'],
                               group_only_regressors=False, keep_regressor_trace=False, p_outlier=0.05)
    return m


In [136]:
def get_full_model_name(m,lag):
    return m if 'nohist' in m else m + '_l' + str(lag)

In [137]:
def run_model(m, mypath, model_name, trace_id, n_samples):
    print("Running {:<s}, trace_id {:2d}".format(model_name,trace_id))
    print("finding starting values")
    try:
        m.find_starting_values() # this should help the sampling
    except Exception as e:
        print(e) #even if starting values couldnt be found, sampling can continue

    print("begin sampling")
    #m.sample(n_samples, burn=n_samples/2, thin=3)
    
    m.sample(n_samples, burn=n_samples/2, thin=3, db='pickle',
        dbname=os.path.join(mypath, model_name, 'modelfit-md%d.db'%trace_id))
    m.save(os.path.join(mypath, model_name, 'modelfit-md%d.model'%trace_id)) # save the model to disk
    
    # ============================================ #
    # save the output values
    # ============================================ #

    # save the DIC for this model
    text_file = open(os.path.join(mypath, model_name, 'DIC-md%d.txt'%trace_id), 'w')
    text_file.write("Model {}: {}\n".format(trace_id, m.dic))
    text_file.close()

    # save the other model comparison indices
    df = dict()
    df['trace_id'] = trace_id
    df['dic_original'] = [m.dic]
    df['aic'] = [aic(m)]
    df['bic'] = [bic(m)]
    df2 = pd.DataFrame(df)
    df2.to_csv(os.path.join(mypath, model_name, 'model_comparison_md%d.csv'%trace_id),index=False)

In [138]:
#The set of models that can be built currently

models_collection = {
    1: 'stimcoding_nohist',  # no history baseline model
    2: 'stimcoding_nohist_svgroup', # no history baseline model
    3: 'stimcoding_dc_resp',  #previous response dependent, nlags needed
    4: 'stimcoding_z_resp',  #previous response dependent, nlags needed
    5: 'stimcoding_dc_z_resp', #previous response dependent, nlags needed
    6: 'stimcoding_dc_stim',  #previous stimulus dependent, nlags needed
    7: 'stimcoding_z_stim',  #previous stimulus dependent, nlags needed
    8: 'stimcoding_dc_z_stim',  #previous stimulus dependent, nlags needed
    9: 'stimcoding_dc_z_st_resp',  #previous response dependent, nlags needed
    10:'stimcoding_dc_z_resp_svgroup',  #previous response dependent, nlags needed
    11:'stimcoding_dc_z_resp_groupsplit',  #previous response dependent, nlags needed
    12:'stimcoding_dc_z_resp_congruency',  #previous response dependent, nlags needed
    13:'regress_dc_resp',  #only previous response factored, nlags needed
    14:'regress_z_resp',  #only previous response factored, nlags needed
    15:'regress_dcz_resp',  #only previous response factored, nlags needed
    16:'regress_dc_resp_stim',  #both previous response fand stimulus actored, nlags needed
    17:'regress_z_resp_stim',  #both previous response fand stimulus actored, nlags needed
    18:'regress_dcz_resp_stim'  #both previous response fand stimulus actored, nlags needed
}


In [139]:
"""
prepare for running.
Setup the folders, select model and the nth prev trial to be analyzed, fetch data
"""
mypath = "output"
datasrc = "~/Documents/Techspace/ddmown/data/coded/pair55.csv"
model = models_collection[8]
lag = 3
chains = 2
trace_ids = range(1,chains+1)
n_samples = 50

full_model_name = get_full_model_name(model,lag)
thispath = os.path.join(mypath,full_model_name)
if not os.path.exists(thispath):
    os.makedirs(thispath)

mydata = hddm.load_csv(datasrc)
mydata.rename(columns={'subject_id':'subj_idx'},inplace=True) #https://groups.google.com/g/hddm-users/c/Dhohjq_U2kU


In [140]:
# ============================================ #
# Main HDDM parameter estimation
# ============================================ #
starttime = time.time()
for trace_id in trace_ids:
    model_filename = os.path.join(mypath, full_model_name, 'modelfit-md%d.model' % trace_id)
    m = make_model(mydata, model, trace_id, lag)
    run_model(m, mypath, full_model_name, trace_id, n_samples)
    elapsed = time.time() - starttime
    print("\nElapsed time for %s, trace_id %d, %d samples: %f seconds\n" % (model, trace_id, n_samples, elapsed))

concat_models(mypath, full_model_name, chains)

Running stimcoding_dc_z_stim_l3, trace_id  1
finding starting values


/Users/sabi/Documents/Techspace/mambaforge/envs/dyadic/lib/python3.5/site-packages/scipy/optimize/optimize.py:1927: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


begin sampling
 [-----------------100%-----------------] 50 of 50 complete in 1.1 sec
Elapsed time for stimcoding_dc_z_stim, trace_id 1, 50 samples: 2.541885 seconds

Running stimcoding_dc_z_stim_l3, trace_id  2
finding starting values
begin sampling
 [-----------------100%-----------------] 50 of 50 complete in 0.9 sec
Elapsed time for stimcoding_dc_z_stim, trace_id 2, 50 samples: 5.154841 seconds

Combining all traces for stimcoding_dc_z_stim_l3
output/stimcoding_dc_z_stim_l3/modelfit-md1.model
output/stimcoding_dc_z_stim_l3/modelfit-md2.model
Performing gelman rubin convergence test

non-convergence found, v_subj.55_2:0.935507447034
non-convergence found, t_subj.55_1:1.27860136891
non-convergence found, t_subj.55_2:1.35502893839
non-convergence found, z_std:1.9303696259
non-convergence found, a_std:0.935822796858
non-convergence found, a_subj.55_1:1.12434987822
non-convergence found, t:1.17362128484
non-convergence found, t_std:1.20494350185
non-convergence found, a:0.951280307384
n

In [141]:
# ============================================ #
# POSTERIOR PREDICTIVES TO ASSESS MODEL FIT
# ============================================ #

starttime = time.time()
print("computing ppc")
# specify how many samples are needed
m = hddm.load(os.path.join(mypath,full_model_name, 'modelfit-combined.model'))
nsmp = 100
ppc = hddm.utils.post_pred_gen(m, append_data=True, samples=nsmp)

# make the csv smaller, save disk space
savecols = list(set(ppc.columns) & set(['rt','rt_sampled', 'response_sampled',
                        'index', 'direction', 'response', 'l1_resp', 'subj_idx']))
ppc = ppc[savecols]
# save as pandas dataframe
ppc.to_csv(os.path.join(mypath, full_model_name, 'ppc_data.csv'), index=True)
elapsed = time.time() - starttime
print( "\nElapsed time for %s, PPC: %f seconds\n" %(full_model_name,elapsed))

computing ppc
 [------------------------137%------------------------] 11 of 8 complete in 41.0 sec
Elapsed time for stimcoding_dc_z_stim_l3, PPC: 42.566101 seconds



In [143]:
# ============================================ #
# QUANTILE OPTIMISATION
# http://ski.clps.brown.edu/hddm_docs/howto.html#run-quantile-opimization
# ============================================ #

subj_params = []
bic = []

for subj_idx, subj_data in mydata.groupby('subj_idx'):
    m_subj = make_model(subj_data, model, 1,lag)
    thismodel = m_subj.optimize('gsquare', quantiles=[0.1, 0.3, 0.5, 0.7, 0.9], n_runs=5)
    thismodel.update({'subj_idx':subj_idx}) # keep original subject number
    subj_params.append(thismodel)
    bic.append(m_subj.bic_info)

params = pd.DataFrame(subj_params)
params.to_csv(os.path.join(mypath, full_model_name, 'Gsquare.csv'))
bic = pd.DataFrame(bic)
bic.to_csv(os.path.join(mypath, full_model_name, 'BIC.csv'))
print("QUANTILE OPTIMISATION. DONE!!!")

/Users/sabi/Documents/Techspace/mambaforge/envs/dyadic/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/sabi/Documents/Techspace/mambaforge/envs/dyadic/lib/python3.5/site-packages/scipy/optimize/optimize.py:1927: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/Users/sabi/Documents/Techspace/mambaforge/envs/dyadic/lib/python3.5/site-packages/scipy/optimize/optimize.py:1928: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1


Optimization terminated successfully.
         Current function value: 514.647160
         Iterations: 15
         Function evaluations: 1737
Optimization terminated successfully.
         Current function value: 514.670952
         Iterations: 16
         Function evaluations: 1861
Optimization terminated successfully.
         Current function value: 538.668771
         Iterations: 10
         Function evaluations: 2048
Optimization terminated successfully.
         Current function value: 515.322621
         Iterations: 8
         Function evaluations: 857
Optimization terminated successfully.
         Current function value: 515.037819
         Iterations: 16
         Function evaluations: 1885
Optimization terminated successfully.
         Current function value: 510.077365
         Iterations: 11
         Function evaluations: 1318
Optimization terminated successfully.
         Current function value: 509.522592
         Iterations: 13
         Function evaluations: 1527
Optimiza